In [1]:
import pyautogui
import time
import random

# Set the duration (in seconds) for which you want the mouse to move
duration = 36000  # 1 hour (you can adjust this value)

# Loop to keep moving the mouse for the specified duration
start_time = time.time()
while time.time() - start_time < duration:
    # Generate random values for x and y coordinates
    move_x = random.randint(-10, 10)  # Adjust the range as needed
    move_y = random.randint(-10, 10)  # Adjust the range as needed

    # Move the mouse with the random values
    pyautogui.move(move_x, move_y)
    
    # Sleep for a short interval to control the mouse movement speed
    time.sleep(random.uniform(0.1, 0.5))  # Adjust the range as needed

# Optional: Return the mouse to its original position
pyautogui.moveTo(0, 0)


FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import re
from nltk.tokenize import sent_tokenize 
from transformers import BertTokenizer, AutoTokenizer
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import transformers
from tqdm import tqdm
import glob
from sklearn.model_selection import train_test_split
import datetime 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import re
from transformers import AutoTokenizer

def tokenize_and_label(text):
    tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

    # Tokenize the text
    tokenized_inputs = tokenizer(text, add_special_tokens=True, truncation=True, padding='max_length', return_tensors="pt")
    tokenized_text = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][0])

    # Initialize label arrays
    section_labels = [0] * len(tokenized_text)
    annotation_labels = [0] * len(tokenized_text)

    # Helper function to assign labels based on spans
    def assign_labels(pattern, label_value, label_array):
        for match in re.finditer(pattern, text):
            start, end = match.span(1)  # Get span of the content inside the annotation
            for i, token in enumerate(tokenized_text):
                token_span = tokenized_inputs.token_to_chars(0, i)
                if token_span is not None:
                    token_start, token_end = token_span
                    # Check if the token is within the span of the annotation
                    if start <= token_start < end or start < token_end <= end:
                        label_array[i] = label_value

    # Assign labels for sections and annotations
    assign_labels(r'\{\#s(.*?)\/\}', 1, section_labels)
    assign_labels(r'\[\#a(.*?)\]', 1, annotation_labels)
    assign_labels(r'\[\#dt(.*?)\]', 2, annotation_labels)
    assign_labels(r'\[\#p(.*?)\]', 3, annotation_labels)

    return tokenized_text, section_labels, annotation_labels

def remove_consecutive_special_tokens(tokenized_text, token_section_labels, token_annotation_labels):
    # Create a list to store the indices of items to delete
    indices_to_delete = []
    tokens_to_drop = {'ns', 'nsc','[', ']', '#', '/', '}', '{'}
    # Iterate through the tokenized_text
    i = 0
    while i < len(tokenized_text):
        token = tokenized_text[i]

        # Check for consecutive special tokens and pattern
        if '[' in token and i + 2 < len(tokenized_text):
            next_tokens = tokenized_text[i + 1:i + 3]
            if '#' in next_tokens and next_tokens[-1] in ['a', 'dt', 'p']:
                indices_to_delete.extend([i, i + 1, i + 2])  # Delete the consecutive tokens
                i += 3  # Move to the token after the consecutive tokens
            else:
                i += 1  # Move to the next token
        elif '{' in token and i + 2 < len(tokenized_text):
            next_tokens = tokenized_text[i + 1:i + 3]
            if '#' in next_tokens and next_tokens[-1] == 's':
                indices_to_delete.extend([i, i + 1, i + 2])  # Delete the consecutive tokens
                i += 3  # Move to the token after the consecutive tokens
            else:
                i += 1  # Move to the next token
        elif token in tokens_to_drop:
            indices_to_delete.append(i)
            i += 1  # Move to the next token
        else:
            i += 1  # Move to the next token

    # Remove items from all three lists simultaneously
    for index in sorted(indices_to_delete, reverse=True):
        del tokenized_text[index]
        del token_section_labels[index]
        del token_annotation_labels[index]

# Example usage
privacy_policy_text = "Your [#ns privacy {#s is important. [#a This section] covers personal data. [#dt Data usage] is critical. /} We [#p value] your trust."
tokenized_text, token_section_labels, token_annotation_labels = tokenize_and_label(privacy_policy_text)

# Call the function to remove consecutive special tokens
remove_consecutive_special_tokens(tokenized_text, token_section_labels, token_annotation_labels)

# Print the processed data for review
for token, sec_label, ann_label in zip(tokenized_text, token_section_labels, token_annotation_labels):
    print(f"{token}: Section Label = {sec_label}, Annotation Label = {ann_label}")

print(len(tokenized_text),len(token_annotation_labels))



Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[CLS]: Section Label = 0, Annotation Label = 0
your: Section Label = 0, Annotation Label = 0
[: Section Label = 0, Annotation Label = 0
privacy: Section Label = 0, Annotation Label = 0
is: Section Label = 1, Annotation Label = 0
important: Section Label = 1, Annotation Label = 0
.: Section Label = 1, Annotation Label = 0
this: Section Label = 1, Annotation Label = 1
section: Section Label = 1, Annotation Label = 1
covers: Section Label = 1, Annotation Label = 0
personal: Section Label = 1, Annotation Label = 0
data: Section Label = 1, Annotation Label = 0
.: Section Label = 1, Annotation Label = 0
data: Section Label = 1, Annotation Label = 2
usage: Section Label = 1, Annotation Label = 2
is: Section Label = 1, Annotation Label = 0
critical: Section Label = 1, Annotation Label = 0
.: Section Label = 1, Annotation Label = 0
we: Section Label = 0, Annotation Label = 0
value: Section Label = 0, Annotation Label = 3
your: Section Label = 0, Annotation Label = 0
trust: Section Label = 0, An

In [ ]:

def tokenize_and_label(text, tokenizer, max_length = 16384):
    tokenized_inputs = tokenizer(
        text, 
        add_special_tokens=True, 
        truncation=True, 
        padding='max_length', 
        max_length=max_length, 
        return_tensors="pt"
    )
    # Tokenize the text
    tokenized_inputs = tokenizer(text, add_special_tokens=True, truncation=True, padding='max_length', max_length=16384, return_tensors="pt")
    tokenized_text = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][0])

    # Initialize label arrays
    section_labels = [-100 if token in ["[CLS]", "[SEP]", "[PAD]"] else 0 for token in tokenized_text]
    annotation_labels = [-100 if token in ["[CLS]", "[SEP]", "[PAD]"] else 0 for token in tokenized_text]


    # Helper function to assign labels based on spans
    def assign_labels(pattern, label_value, label_array):
        for match in re.finditer(pattern, text):
            start, end = match.span(1)  # Get span of the content inside the annotation
            for i, token in enumerate(tokenized_text):
                token_span = tokenized_inputs.token_to_chars(0, i)
                if token_span is not None:
                    token_start, token_end = token_span
                    # Check if the token is within the span of the annotation
                    if start <= token_start < end or start < token_end <= end:
                        label_array[i] = label_value

    # Assign labels for sections and annotations
    assign_labels(r'\{\#s(.*?)\/\}', 1, section_labels)
    assign_labels(r'\[\#a(.*?)\]', 1, annotation_labels)
    assign_labels(r'\[\#dt(.*?)\]', 2, annotation_labels)
    assign_labels(r'\[\#p(.*?)\]', 3, annotation_labels)

    return tokenized_text, section_labels, annotation_labels

def load_and_process_data(root_dir):
    all_annotated_data = []
    full_text = ""

    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file == 'privacy_policy.txt':
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as file:
                    text = file.read()
                    full_text += text + " "  # Append text from each file
                    sections = re.findall(r'\{\#s(.*?)\/\}', text, re.DOTALL)
                    for section in sections:
                        actions = re.findall(r'\[\#a(.*?)\]', section)
                        data_types = re.findall(r'\[\#dt(.*?)\]', section)
                        purposes = re.findall(r'\[\#p(.*?)\]', section)
                        all_annotated_data.append({'section': section, 'actions': actions, 'data_types': data_types, 'purposes': purposes})
    
    return full_text, all_annotated_data

# Function to remove consecutive special tokens
def remove_tokens(tokenized_text, token_section_labels, token_annotation_labels):
    # Create a list to store the indices of items to delete
    indices_to_delete = []
    tokens_to_drop = {'ns', 'nsc','[', ']', '#', '/', '}', '{'}
    # Iterate through the tokenized_text
    i = 0
    while i < len(tokenized_text):
        token = tokenized_text[i]

        # Check for consecutive special tokens and pattern
        if '[' in token and i + 2 < len(tokenized_text):
            next_tokens = tokenized_text[i + 1:i + 3]
            if '#' in next_tokens and next_tokens[-1] in ['a', 'dt', 'p']:
                indices_to_delete.extend([i, i + 1, i + 2])  # Delete the consecutive tokens
                i += 3  # Move to the token after the consecutive tokens
            else:
                i += 1  # Move to the next token
        elif '{' in token and i + 2 < len(tokenized_text):
            next_tokens = tokenized_text[i + 1:i + 3]
            if '#' in next_tokens and next_tokens[-1] == 's':
                indices_to_delete.extend([i, i + 1, i + 2])  # Delete the consecutive tokens
                i += 3  # Move to the token after the consecutive tokens
            else:
                i += 1  # Move to the next token
        elif token in tokens_to_drop:
            indices_to_delete.append(i)
            i += 1  # Move to the next token
        else:
            i += 1  # Move to the next token

    # Remove items from all three lists simultaneously
    for index in sorted(indices_to_delete, reverse=True):
        del tokenized_text[index]
        del token_section_labels[index]
        del token_annotation_labels[index]


tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

root_dir = 'policies_annotated/annotated_policies'

# Load and process data
privacy_policy_text, annotated_data = load_and_process_data(root_dir)

# Tokenize and label the data
tokenized_text, token_section_labels, token_annotation_labels = tokenize_and_label(privacy_policy_text, tokenizer)


# Remove consecutive special tokens
remove_tokens(tokenized_text, token_section_labels, token_annotation_labels)

# Pad tokenized text and labels to the seq length
max_length = 512

def pad_sequence(sequence, desired_length, tokenizer):
    pad_token_id = tokenizer.pad_token_id  # Get the special pad token ID from the tokenizer
    return sequence + [pad_token_id] * (desired_length - len(sequence))

tokenized_text = pad_sequence(tokenized_text, max_length, tokenizer)
token_section_labels = pad_sequence(token_section_labels, max_length, tokenizer)
token_annotation_labels = pad_sequence(token_annotation_labels, max_length, tokenizer)

def replace_integers_with_special_tag(tokenized_text, special_tag="[PAD]"):
    return [special_tag if isinstance(token, int) else token for token in tokenized_text]

# Replace any integer tokens with a special tag
tokenized_text = replace_integers_with_special_tag(tokenized_text)



# Print a portion of the processed data for review
for token, sec_label, ann_label in zip(tokenized_text, token_section_labels, token_annotation_labels):
    print(f"{token}: Section Label = {sec_label}, Annotation Label = {ann_label}")

print(len(privacy_policy_text))
print(len(tokenized_text), len(token_section_labels), len(token_annotation_labels))




[CLS]: Section Label = -100, Annotation Label = -100
[: Section Label = 0, Annotation Label = 0
introduction: Section Label = 0, Annotation Label = 0
at: Section Label = 0, Annotation Label = 0
st: Section Label = 0, Annotation Label = 0
##fm: Section Label = 0, Annotation Label = 0
##stud: Section Label = 0, Annotation Label = 0
##io: Section Label = 0, Annotation Label = 0
,: Section Label = 0, Annotation Label = 0
we: Section Label = 0, Annotation Label = 0
value: Section Label = 0, Annotation Label = 0
your: Section Label = 0, Annotation Label = 0
privacy: Section Label = 0, Annotation Label = 0
and: Section Label = 0, Annotation Label = 0
are: Section Label = 0, Annotation Label = 0
dedicated: Section Label = 0, Annotation Label = 0
to: Section Label = 0, Annotation Label = 0
safe: Section Label = 0, Annotation Label = 0
##guard: Section Label = 0, Annotation Label = 0
##ing: Section Label = 0, Annotation Label = 0
your: Section Label = 0, Annotation Label = 0
personal: Section La

In [ ]:

class PrivacyPolicyDataset(Dataset):
    def __init__(self, input_ids, section_labels, annotation_labels):
        self.input_ids = input_ids
        self.section_labels = section_labels
        self.annotation_labels = annotation_labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.input_ids[idx] != tokenizer.pad_token_id,
            'section_labels': self.section_labels[idx],
            'annotation_labels': self.annotation_labels[idx]
        }
# Tokenize and Label the Data (Assuming this step is already correctly done)
tokenized_text, token_section_labels, token_annotation_labels = tokenize_and_label(privacy_policy_text, tokenizer)

# Convert tokenized text to token IDs
input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)

# Split the data into sequences
def split_into_sequences(data, sequence_length):
    return [data[i:i + sequence_length] for i in range(0, len(data), sequence_length)]

input_ids_split = split_into_sequences(input_ids, sequence_length=512)
section_labels_split = split_into_sequences(token_section_labels, sequence_length=512)
annotation_labels_split = split_into_sequences(token_annotation_labels, sequence_length=512)

# Convert lists to tensors
input_ids_tensor = torch.tensor(input_ids_split)
section_labels_tensor = torch.tensor(section_labels_split)
annotation_labels_tensor = torch.tensor(annotation_labels_split)

# Create dataset and dataloader
dataset = PrivacyPolicyDataset(input_ids_tensor, section_labels_tensor, annotation_labels_tensor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Debugging: Check dataset and dataloader
print(f"Dataset size: {len(dataset)} sequences")
print(f"Total batches per epoch: {len(dataloader)}")



Dataset size: 32 sequences
Total batches per epoch: 2
{'input_ids': tensor([[ 2368, 30107,  1352,  ...,   198, 13789, 12369],
        [  776,   145,   139,  ...,  3000,   106,  4456],
        [  102,   260,  3000,  ...,  1967,   260,  3000],
        ...,
        [  111,  2368, 11441,  ...,  9108,  3034,   626],
        [  190,   106,  9408,  ...,   112,   603,  3034],
        [ 4395,   843,   131,  ...,   260,  3000,   118]]), 'attention_mask': tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]]), 'section_labels': tensor([[   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    1,    1,    1],
        [-100,    0,    0,  ...,    1,    1,    1],
        ...,
        [   0,    0,    0,  ...,    0

In [ ]:
from transformers import BertModel, BertConfig, BertPreTrainedModel
import torch.nn as nn

class BertForTokenAndSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.num_token_labels = 4  # Adjust this based on your needs

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

        # Token classification head
        self.token_classifier = nn.Linear(config.hidden_size, self.num_token_labels)
        
        # Sequence classification head
        self.seq_classifier = nn.Linear(config.hidden_size, self.num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, 
                position_ids=None, head_mask=None, labels=None, token_labels=None):

        outputs = self.bert(input_ids, attention_mask=attention_mask,
                            token_type_ids=token_type_ids, position_ids=position_ids, 
                            head_mask=head_mask)

        sequence_output = self.dropout(outputs[0])
        pooled_output = self.dropout(outputs[1])

        # Token-level classification
        token_logits = self.token_classifier(sequence_output)

        # Sequence-level classification
        sequence_logits = self.seq_classifier(pooled_output)

        return token_logits, sequence_logits

# Load the configuration from 'bert-base-uncased'
config = BertConfig.from_pretrained('bert-base-uncased')
config.num_labels = 2  # Adjust based on your sequence classification needs

# Create an instance of your model
model = BertForTokenAndSequenceClassification(config)


In [6]:
from transformers import BertModel, BertConfig, BertPreTrainedModel
import torch
import torch.nn as nn

class LoRALayer(nn.Module):
    def __init__(self, config, rank=4):
        super().__init__()
        self.rank = rank
        self.attention_lora = nn.Linear(config.hidden_size, config.hidden_size, bias=False)
        self.output_lora = nn.Linear(config.hidden_size, config.hidden_size, bias=False)

    def forward(self, hidden_states):
        attention_delta = self.attention_lora(hidden_states)
        output_delta = self.output_lora(hidden_states)
        return hidden_states + attention_delta, hidden_states + output_delta

class BertForTokenAndSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.num_token_labels = 4  # Adjust this based on your needs

        self.bert = BertModel(config)
        self.lora = LoRALayer(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.token_classifier = nn.Linear(config.hidden_size, self.num_token_labels)
        self.seq_classifier = nn.Linear(config.hidden_size, self.num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, 
                position_ids=None, head_mask=None, labels=None, token_labels=None):

        outputs = self.bert(input_ids, attention_mask=attention_mask,
                            token_type_ids=token_type_ids, position_ids=position_ids, 
                            head_mask=head_mask)

        # Apply LoRA layer to the sequence output
        sequence_output, _ = self.lora(outputs[0])
        sequence_output = self.dropout(sequence_output)

        # Apply LoRA layer to the pooled output
        _, pooled_output = self.lora(outputs[1])
        pooled_output = self.dropout(pooled_output)

        # Token-level classification
        token_logits = self.token_classifier(sequence_output)

        # Sequence-level classification
        sequence_logits = self.seq_classifier(pooled_output)

        return token_logits, sequence_logits

# Load the configuration from 'bert-base-uncased'
config = BertConfig.from_pretrained('bert-base-uncased')
config.num_labels = 2  # Adjust based on your sequence classification needs

# Create an instance of your model
model = BertForTokenAndSequenceClassification(config)


In [ ]:
# Load pre-trained BERT model

checkpoint_dir = './checkpoint'
os.makedirs(checkpoint_dir, exist_ok=True)
# Assuming the model is already defined and loaded
# ...

# Define optimizer and loss functions
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
token_loss_fn = nn.CrossEntropyLoss()
sequence_loss_fn = nn.CrossEntropyLoss()

num_epochs = 100
print(f"Dataset size: {len(dataset)} sequences")

# Check the DataLoader
print(f"Batch size: {dataloader.batch_size}")
print(f"Total batches per epoch: {len(dataloader)}")
# Training loop
# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss_accumulated = 0  # To accumulate loss for reporting

    for batch in dataloader:
        optimizer.zero_grad()

        # Ensure tensors are on the same device as the model
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        token_labels = batch['annotation_labels'].to(model.device)
        section_labels = batch['section_labels'].to(model.device)

        # Forward pass
        token_logits, sequence_logits = model(input_ids, attention_mask=attention_mask)

        # Compute losses
        token_loss = token_loss_fn(
            token_logits.view(-1, model.num_token_labels), 
            token_labels.view(-1)
        )
        
        # Make sure section_labels is a 1D tensor with size [batch_size]
        if section_labels.dim() > 1:
            # Assuming the label for a sequence is the first element in section_labels
            section_labels = section_labels[:, 0]

        sequence_loss = sequence_loss_fn(
            sequence_logits.view(-1, model.num_labels), 
            section_labels
        )

        # Combine losses
        total_loss = token_loss + sequence_loss
        total_loss_accumulated += total_loss.item()

        # Backward pass and optimization
        total_loss.backward()
        optimizer.step()

        print(f"Epoch {epoch}, Batch Loss: {total_loss.item()}")

    average_loss = total_loss_accumulated / len(dataloader)
    print(f"Epoch {epoch}, Average Loss: {average_loss}")

    #Rewrite each epoch 
    model_save_path = os.path.join(checkpoint_dir, 'berts1.0.model_epoch_{}.pt'.format(epoch))
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")


Dataset size: 4 sequences
Batch size: 16
Total batches per epoch: 1
Epoch 0, Batch Loss: 2.7629594802856445
Epoch 0, Average Loss: 2.7629594802856445
Model saved to ./checkpoint\berts1.0.model_epoch_0.pt
Epoch 1, Batch Loss: 1.442682147026062
Epoch 1, Average Loss: 1.442682147026062
Model saved to ./checkpoint\berts1.0.model_epoch_1.pt
Epoch 2, Batch Loss: 1.3400335311889648
Epoch 2, Average Loss: 1.3400335311889648
Model saved to ./checkpoint\berts1.0.model_epoch_2.pt
Epoch 3, Batch Loss: 1.1738572120666504
Epoch 3, Average Loss: 1.1738572120666504
Model saved to ./checkpoint\berts1.0.model_epoch_3.pt
Epoch 4, Batch Loss: 0.7516119480133057
Epoch 4, Average Loss: 0.7516119480133057
Model saved to ./checkpoint\berts1.0.model_epoch_4.pt
Epoch 5, Batch Loss: 0.5989213585853577
Epoch 5, Average Loss: 0.5989213585853577
Model saved to ./checkpoint\berts1.0.model_epoch_5.pt
Epoch 6, Batch Loss: 0.6881771087646484
Epoch 6, Average Loss: 0.6881771087646484
Model saved to ./checkpoint\berts1.0

RuntimeError: [enforce fail at inline_container.cc:424] . unexpected pos 230867776 vs 230867664

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from sklearn.model_selection import train_test_split
from torch.utils.data.dataloader import default_collate


# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

# Split the data into training and validation sets first
train_texts, val_texts, train_section_labels, val_section_labels, train_annotation_labels, val_annotation_labels = train_test_split(
    tokenized_text, token_section_labels, token_annotation_labels, test_size=0.1
)
print(len(train_texts), len(train_section_labels), len(train_annotation_labels))
# Debugging code
for idx, labels in enumerate(train_annotation_labels[:5]):
    if not isinstance(labels, (list, tuple)):
        print(f"Index {idx} in train_annotation_labels is not a list or tuple: {labels}")

# Similar check for val_annotation_labels if necessary


# Define the encode_data function
def encode_data(tokenizer, texts):
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=512, return_tensors="pt")
    return encodings

train_texts = [text for text in train_texts if isinstance(text, str) and text]
val_texts = [text for text in val_texts if isinstance(text, str) and text]

# Encode the training and validation data
train_encodings = encode_data(tokenizer, train_texts)
val_encodings = encode_data(tokenizer, val_texts)

# Custom Dataset class

class PrivacyPolicyDataset(Dataset):
    def __init__(self, encodings, section_labels, annotation_labels):
        self.encodings = encodings
        self.section_labels = section_labels
        self.annotation_labels = annotation_labels

    def __getitem__(self, idx):
        item = {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'section_label': torch.tensor(self.section_labels[idx], dtype=torch.long),
            'annotation_labels': torch.tensor(self.annotation_labels[idx], dtype=torch.long)
        }
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])




def pad_labels(labels, max_length, pad_value=-100):
    # Ensure labels are padded to max_length
    return labels + [pad_value] * (max_length - len(labels))
class PrivacyPolicyDataCollator:
    def __call__(self, batch):
        collated_batch = default_collate(batch)

        # If your labels are already tensors in the dataset, you may not need additional processing here
        # However, ensure that they are of the correct shape and type for your model

        return collated_batch




# Assuming you're focusing on one set of labels, e.g., section_labels

train_dataset = PrivacyPolicyDataset(train_encodings, train_section_labels, train_annotation_labels)
val_dataset = PrivacyPolicyDataset(val_encodings, val_section_labels, val_annotation_labels)


from transformers import AutoModel
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self, base_model, num_labels_section, num_labels_annotation):
        super().__init__()
        self.base_model = AutoModel.from_pretrained(base_model)
        # Classifier for section labels (sequence-level classification)
        self.classifier_section = nn.Linear(self.base_model.config.hidden_size, num_labels_section)
        # Classifier for annotation labels (token-level classification)
        self.classifier_annotation = nn.Linear(self.base_model.config.hidden_size, num_labels_annotation)

    def forward(self, input_ids, attention_mask):
        # Forward pass through the base model
        outputs = self.base_model(input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]

        # Sequence-level classification
        section_logits = self.classifier_section(sequence_output[:, 0, :])  # Use the [CLS] token representation

        # Token-level classification
        annotation_logits = self.classifier_annotation(sequence_output)

        return section_logits, annotation_logits


# Instantiate the model
num_labels_section = 2  # Adjust as per your dataset
num_labels_annotation = 4  # Adjust as per your dataset
model = CustomModel('allenai/scibert_scivocab_uncased', num_labels_section, num_labels_annotation)


'''# Define the model
model = AutoModelForTokenClassification.from_pretrained('allenai/scibert_scivocab_uncased', num_labels=4) # num_labels should be the number of unique labels

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,  # Reduced batch size
    gradient_accumulation_steps=2,  # Use if reducing batch size impacts performance
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    # Other arguments...
)

data_collator = DataCollatorForTokenClassification(tokenizer)


# Make sure to set the model to the appropriate device
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()

# Optionally, evaluate the model
eval_results = trainer.evaluate()
print(eval_results) '''

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

# Assuming model, train_dataset, and val_dataset are already defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Loss functions
loss_fn_section = torch.nn.CrossEntropyLoss()
loss_fn_annotation = torch.nn.CrossEntropyLoss()

data_collator = PrivacyPolicyDataCollator()
train_loader = DataLoader(train_dataset, batch_size=24, shuffle=True, collate_fn=PrivacyPolicyDataCollator())
val_loader = DataLoader(val_dataset, batch_size=24, collate_fn=PrivacyPolicyDataCollator())

for i, batch in enumerate(train_loader):
    print(f"Batch {i} loaded")
    if i >= 1:  # Just to check the first couple of batches
        break



# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()

        # Move batch to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        section_labels = batch['section_label'].to(device)
        annotation_labels = batch['annotation_labels'].to(device)

        print("Section labels shape:", section_labels.shape)
        print("Annotation labels shape:", annotation_labels.shape)


        # Forward pass
        section_logits, annotation_logits = model(input_ids, attention_mask)

        # Compute loss for section labels
        loss_section = loss_fn_section(section_logits, section_labels)

        # Check and reshape annotation_labels
        assert annotation_labels.shape == (input_ids.size(0), input_ids.size(1))  # Ensure correct shape
        annotation_labels_flat = annotation_labels.view(-1)

        # Compute loss for annotation labels
        loss_annotation = loss_fn_annotation(annotation_logits.view(-1, num_labels_annotation), annotation_labels_flat)

        # Total loss
        loss = loss_section + loss_annotation

        loss.backward()
        optimizer.step()

    # ... Rest of your training loop ...


    # Validation



    # Validation step (optional)
    model.eval()
    total_val_loss = 0
    total_val_loss_section = 0
    total_val_loss_annotation = 0
    with torch.no_grad():
        for batch in val_loader:
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            section_labels = batch['section_labels'].to(device)
            annotation_labels = batch['annotation_labels'].to(device)

            # Forward pass
            section_logits, annotation_logits = model(input_ids, attention_mask)

            # Compute loss
            val_loss_section = loss_fn_section(section_logits.view(-1, num_labels_section), section_labels.view(-1))
            val_loss_annotation = loss_fn_annotation(annotation_logits.view(-1, num_labels_annotation), annotation_labels.view(-1))
            val_loss = val_loss_section + val_loss_annotation

            # Accumulate the validation loss
            total_val_loss += val_loss.item()
            total_val_loss_section += val_loss_section.item()
            total_val_loss_annotation += val_loss_annotation.item()
            # Optionally accumulate other metrics here

    # Calculate the average validation loss over all batches
    avg_val_loss = total_val_loss / len(val_loader)
    avg_val_loss_section = total_val_loss_section / len(val_loader)
    avg_val_loss_annotation = total_val_loss_annotation / len(val_loader)
    # Optionally calculate average for other metrics

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {loss.item()}, Validation Loss: {avg_val_loss}, Section Val Loss: {avg_val_loss_section}, Annotation Val Loss: {avg_val_loss_annotation}")
    # Optionally print other metrics

# Save the model after training
torch.save(model.state_dict(), 'results/dsbert.pt')



NameError: name 'tokenized_text' is not defined

In [ ]:
def predict(text, model, tokenizer, label_mapping):
    # Tokenize the input text
    inputs = tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="pt")

    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted label IDs
    logits = outputs.logits
    predicted_label_ids = torch.argmax(logits, dim=2)

    # Decode label IDs to labels
    labels = [label_mapping.get(id.item(), 'Unknown Label') for id in predicted_label_ids[0].cpu().numpy()]

    # Remove padding labels (if any)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    pairs = list(zip(tokens, labels))
    return [(token, label) for token, label in pairs if label != -100 and label != 'Unknown Label']

# Example usage
text = "If we learn that we have collected personal information from a child under 13, we will take appropriate steps to delete the information. 7. Changes to this Privacy Policy We may update this Privacy Policy from time to time to reflect changes in our practices or for other operational, legal, or regulatory reasons. We will notify you of any changes by posting the updated Privacy Policy on the App. Your continued use of the App after the changes become effective constitutes your acceptance of the updated Privacy Policy. 8. Contact Information If you have any questions or concerns about this Privacy Policy or the App's privacy practices, please contact us at noblemancompany99@gmail.com By using the Dream Now App, you acknowledge that you have read, understood, and agree to this Privacy Policy. Page updated"
prediction = predict(text, model, tokenizer, label_mapping)

# Print the tokens with their predicted labels
for token, label in prediction:
    print(f"{token}: {label}")

def predict_and_annotate(text, model, tokenizer, label_mapping):
    # Tokenize the input text
    inputs = tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="pt")

    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted label IDs
    logits = outputs.logits
    predicted_label_ids = torch.argmax(logits, dim=2)

    # Decode label IDs to labels
    labels = [label_mapping.get(id.item(), 'Unknown Label') for id in predicted_label_ids[0].cpu().numpy()]

    # Tokenize the text to match with labels
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    # Reconstruct the text with annotations
    annotated_text = ""
    for token, label in zip(tokens, labels):
        if token.startswith("##"):
            annotated_text += token[2:]
        else:
            annotated_text += " " + token

        if label not in ['-100', 'O', 'Unknown Label']:
            annotated_text += f"<{label}>"

    return annotated_text.strip()

# Example usage
annotated_text = predict_and_annotate(text, model, tokenizer, label_mapping)
print(annotated_text)



[CLS] if we learn that we have collected personal information from a child under 13 , we will take appropriate steps to delete the information . 7 . changes to this privacy policy we may update this privacy policy from time to time to reflect changes in our practices or for other operational , legal , or regulatory reasons . we will notify you of any changes by posting the updated privacy policy on the app . your continued use of the app after the changes become effective constitutes your acceptance of the updated privacy policy . 8 . contact information if you have any questions or concerns about this privacy policy or the app ' s privacy practices , please contact us at noble [SEP]


In [ ]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

# Load the trained model and tokenizer
model_path = 'results/checkpoint-7000'
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModelForTokenClassification.from_pretrained(model_path)
model.eval()

# Load label mapping
label_mapping = {0: 'O', 1: 'B-ACTION', 2: 'I-ACTION', 3: 'B-DATATYPE', 4: 'I-DATATYPE', 5: 'B-PURPOSE', 6: 'I-PURPOSE'}
# Add additional labels if your model was trained with more labels

# Function to add new labels to the input text
def annotate_text(text):
    # Tokenize input text
    tokenized_inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding='max_length')
    input_ids = tokenized_inputs["input_ids"]

    # Predict labels
    with torch.no_grad():
        outputs = model(input_ids)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    # Process each token and its predicted label
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    new_labels = [label_mapping.get(label_id.item(), 'Unknown') for label_id in predictions[0]]

    # Reconstruct the text with new labels
    annotated_text = ""
    for token, label in zip(tokens, new_labels):
        if token.startswith("##"):
            annotated_text += token[2:]  # Append without space for subword tokens
        else:
            annotated_text += " " + token  # Append with space
        if label != 'O' and label != 'Unknown':
            annotated_text += f"[{label}]"

    return annotated_text.strip()

# Example usage
input_text = "The terms used in this Privacy Policy have the same meanings as in our Terms and Conditions, which is accessible at www.BERTinc.org, unless otherwise defined in this Privacy Policy. Information Collection and Use For a better experience while using our Service, we may require you to provide us with certain personally identifiable information, including but not limited to your name, phone number, and postal address. The information that we collect will be used to contact or identify you. Log Data We want to inform you that whenever you visit our Service, we collect information that your browser sends to us that is called Log Data. This Log Data may include information such as your computer’s Internet Protocol (“IP”) address, browser version, pages of our Service that you visit, the time and date of your visit, the time spent on those pages, and other statistics. Cookie Cookies are files with small amount of data that is commonly used an anonymous unique identifier. These are sent to your browser from the website that you visit and are stored on your computer’s hard drive. Our website uses these “cookies” to collection information and to improve our Service. You have the option to either accept or refuse these cookies, and know when a cookie is being sent to your computer. If you choose to refuse our cookies, you may not be able to use some portions of our Service."
annotated_text = annotate_text(input_text)
print("Annotated Text:", annotated_text)


Annotated Text: [CLS] the terms[B-DATATYPE] used[I-PURPOSE] in this privacy policy have the same meanings as in our terms and conditions ,[B-DATATYPE] which is accessible at www . bertinc . org , unless otherwise defined[B-DATATYPE] in[I-PURPOSE] this privacy policy . information collection[I-PURPOSE] and[I-PURPOSE] use for a better experience while using our service , we may require you to provide us with[B-DATATYPE] certain[I-PURPOSE] personally identifiable information ,[B-DATATYPE] including[I-PURPOSE] but not limited to your name , phone number ,[B-DATATYPE] and postal[I-PURPOSE] address . the information that we collect will be used to contact or identify you[I-PURPOSE] . log data we want to[B-DATATYPE] inform[I-PURPOSE] you that whenever you visit our service[I-PURPOSE] , we collect information that your browser sends to us that is[B-DATATYPE] called log[B-DATATYPE] data . this log data may include information such[I-PURPOSE] as your computer ’ s internet protocol ( “ ip ” ) add

In [ ]:
class PrivacyPolicyDataset(Dataset):
    def __init__(self, encodings, section_labels, annotation_labels):
        self.encodings = encodings
        self.section_labels = section_labels
        self.annotation_labels = annotation_labels

    def __getitem__(self, idx):
        item = {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.section_labels[idx])  # Using section labels for demonstration
        }
        return item

    def __len__(self):
        return len(self.section_labels)

full_text, _ = load_and_process_data(root_dir)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

# Tokenize and label the data (simplified for demonstration)
inputs = tokenizer(full_text, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
# Tokenize and label the data (simplified for demonstration)
# Tokenize and label the data (simplified for demonstration)
input_ids_list = inputs.input_ids.tolist()  # Convert input_ids to a list of lists
labels = [torch.zeros(len(input_ids), dtype=torch.long) for input_ids in input_ids_list]  # Dummy labels for demonstration

# Split data into train and validation sets for each input sequence
train_inputs, val_inputs, train_labels, val_labels = [], [], [], []

for input_ids, label in zip(input_ids_list, labels):
    # Split each input sequence and its label into train and validation sets
    split_input_ids = input_ids
    split_label = label

    # Append to the respective train and validation lists
    if len(split_input_ids) > 1:
        train_inputs.append(split_input_ids)
        train_labels.append(split_label)
    else:
        val_inputs.append(split_input_ids)
        val_labels.append(split_label)

# Create datasets
train_dataset = PrivacyPolicyDataset(train_texts, train_labels, train_labels)  # Use different labels for section and annotation
val_dataset = PrivacyPolicyDataset(val_texts, val_labels, val_labels)  # Use different labels for section and annotation

In [ ]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForTokenClassification, AdamW
from transformers import get_linear_schedule_with_warmup

# Initialize the model and tokenizer
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=4)  # 4 labels: Section, Action, Data Type, Purpose

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataset))

# Define training parameters
epochs = 3

# Define batch size
batch_size = 8

# Create DataLoader objects
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)

print(train_dataset[:1])
# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
    
    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_train_loss:.4f}")

# Evaluation loop
model.eval()
total_eval_loss = 0.0

for batch in val_dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    
    loss = outputs.loss
    total_eval_loss += loss.item()

avg_eval_loss = total_eval_loss / len(val_dataloader)
print(f"Average Validation Loss: {avg_eval_loss:.4f}")


Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

TypeError: list indices must be integers or slices, not str

In [10]:
import torch
from transformers import BertTokenizer

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the model
model_path = './checkpoint/berts1.0.model_epoch_6.pt'  # Assuming num_epochs is defined
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode


def preprocess(text, tokenizer, max_length=512):
    inputs = tokenizer.encode_plus(
        text, 
        add_special_tokens=True, 
        return_tensors='pt', 
        max_length=max_length, 
        truncation=True,
        padding='max_length',
        return_attention_mask=True
    )
    return inputs

# Example text
text = "While using Our Service, We may ask You to provide Us with certain personally identifiable information that can be used to contact or identify You. Personally identifiable information may include, but is not limited to: Email address Usage Data Usage Data Usage Data is collected automatically when using the Service. Usage Data may include information such as Your Device's Internet Protocol address (e.g. IP address), browser type, browser version, the pages of our Service that You visit, the time and date of Your visit, the time spent on those pages, unique device identifiers and other diagnostic data. When You access the Service by or through a mobile device, We may collect certain information automatically, including, but not limited to, the type of mobile device You use, Your mobile device unique ID, the IP address of Your mobile device, Your mobile operating system, the type of mobile Internet browser You use, unique device identifiers and other diagnostic data.  We may also collect information that Your browser sends whenever You visit our Service or when You access the Service by or through a mobile device."
text = "Your example text goes here."
inputs = preprocess(text, tokenizer)



with torch.no_grad():
    token_logits, sequence_logits = model(
        inputs['input_ids'], 
        attention_mask=inputs['attention_mask']
    )

# Get predictions
token_predictions = torch.argmax(token_logits, dim=-1).squeeze().tolist()
sequence_predictions = torch.argmax(sequence_logits, dim=-1).squeeze().tolist()


# Decode token predictions back to labels (assuming you have a label map)
# Assuming you have mappings for your labels
token_label_map = {0: "TokenLabel1", 1: "TokenLabel2", 2: "TokenLabel3", 3: "TokenLabel4"}
sequence_label_map = {0: "SectionLabel1", 1: "SectionLabel2"}

# Decode token predictions
decoded_token_predictions = [token_label_map[label] for label in token_predictions]

# Tokenize input text for alignment
tokens = tokenizer.tokenize(tokenizer.decode(inputs['input_ids'][0]))

# Align tokens with predictions
for token, label in zip(tokens, decoded_token_predictions):
    print(f"{token}: {label}")

# Decode sequence prediction
decoded_sequence_prediction = sequence_label_map[sequence_predictions]
print(f"Sequence Label: {decoded_sequence_prediction}")







[CLS]: TokenLabel1
your: TokenLabel1
example: TokenLabel1
text: TokenLabel1
goes: TokenLabel1
here: TokenLabel1
.: TokenLabel1
[SEP]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1
[PAD]: TokenLabel1